In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd
import numpy as np

In [3]:
# %pip install --q vectice -U
import vectice as vect

vec = vect.connect(config="tut.json")

VECTICE_API_ENDPOINT is deprecated and will be removed in 23.3.1.0, please use VECTICE_HOST instead.
Welcome, Aidan. You`re now successfully connected to Vectice.

To access your personal workspace, use connection.my_workspace
To access a specific workspace, use connection.workspace(Workspace ID)
To get a list of workspaces you can access and their IDs, use connection.list_workspaces()

If you are using a notebook you can call the help by using a Vectice returned object with the builtin notebook "?":
>> connection?

If you are using an IDE you can call the help() method on any object returned by Vectice:
>> help(connection)

For quick access to the list of workspaces in the Vectice web app, visit:
https://dev.vectice.com/workspaces


In [4]:
workspace = vec.workspace("Create Sample Tutorial Project") # Your personal workspace can be access with my_workspace property

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Workspace 'Create Sample Tutorial Project' successfully retrieved."

For quick access to the workspace in the Vectice web app, visit:
https://dev.vectice.com/browse/workspace/WSP-273


In [5]:
project = workspace.project("Store Sales Prediction") # pass the ID of your tutorial project

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Project 'Store Sales Prediction' successfully retrieved."

For quick access to the Project in the Vectice web app, visit:
https://dev.vectice.com/browse/project/PRJ-561


In [6]:
prep_iteration = project.phase("PHA-1170").create_iteration()

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Phase 'Data Preparation' successfully retrieved."

For quick access to the Phase in the Vectice web app, visit:
https://dev.vectice.com/browse/phase/PHA-1170
New Iteration number '3' created.

For quick access to the Iteration in the Vectice web app, visit:
https://dev.vectice.com/browse/iteration/ITR-331


In [7]:
df_initial = pd.read_csv("SampleSuperstore.csv")

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
origin_ds = vect.FileResource(paths="SampleSuperstore.csv", dataframes=df_initial)


origin_dataset = vect.Dataset.origin(
    name="SampleSuperstore origin dataset",
    resource=origin_ds, 
)

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
File: SampleSuperstore.csv wrapped successfully.


In [9]:
prep_iteration.step_select_data = origin_dataset

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
New Version: 'Version 3' of Dataset: 'SampleSuperstore origin dataset' added to Step: Select Data
Attachments: None
Link to Step: https://dev.vectice.com/browse/iteration/ITR-331



In [10]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    #Reducing Cardinality
    top_ten_cities=df["City"].value_counts().head(10).index
    df["City"]=df["City"].apply(lambda c: c if c in top_ten_cities else "others")
    top_ten_states=df["State"].value_counts().head(10).index
    df["State"]=df["State"].apply(lambda c: c if c in top_ten_states else "others")
    
    ## Dealing with Outliers
    q1,q2 =df["Profit"].quantile([0.1,0.9])
    df=df[df["Profit"].between(q1,q2)]
    
    
    return df

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df = wrangle("SampleSuperstore.csv")
df.describe()

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Postal Code,Sales,Quantity,Discount,Profit
count,7994.000000,7994.000000,7994.000000,7994.000000,7994.000000
mean,55874.241931,98.123799,3.558419,0.131893,15.795750
std,32393.876810,178.131556,2.083735,0.182274,20.670667
min,1040.000000,0.444000,1.000000,0.000000,-15.466600
25%,23434.000000,14.016000,2.000000,0.000000,2.957400
50%,60035.000000,36.328000,3.000000,0.000000,8.666500
75%,90032.000000,105.520000,5.000000,0.200000,22.200000
max,99301.000000,4164.050000,14.000000,0.800000,89.222400


In [12]:
df.to_csv("Prepared Dataset.csv")

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
prepared_ds = vect.FileResource(paths="SampleSuperstore.csv", dataframes=df)


prepared_ds = vect.Dataset.clean(
    name="SampleSuperstore Prepared Dataset",
    resource=prepared_ds,
    derived_from=origin_dataset
)

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
File: SampleSuperstore.csv wrapped successfully.


In [14]:
prep_iteration.step_clean_data = prepared_ds 

C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
New Version: 'Version 2' of Dataset: 'SampleSuperstore Prepared Dataset' added to Step: Clean data
Attachments: None
Link to Step: https://dev.vectice.com/browse/iteration/ITR-331

